## **Exercise 04: enrichment and transformations**

Prepare the project:

In [1]:
%pip install --upgrade pip
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



Import allowed modules:

In [2]:
import warnings  # For ignoring warnings

import numpy as np
import pandas as pd

In [3]:
warnings.filterwarnings("ignore", )  # Ignore warnings

### read the *JSON* file that you saved in `ex02`:

* one of the columns has the `float` type, so let us define the format of it in *pandas* using `pd.options.display.float_format`: floats should be displayed with two decimals:

In [4]:
pd.options.display.float_format = "{:.2f}".format

Create a dictionary for `read_json()` method calling:

In [5]:
read_json_params: dict = {"file": "auto.json", "file_path": "../../datasets/", }

Read the data file to *Pandas* dataframe:

In [6]:
df: pd.DataFrame = pd.read_json(read_json_params["file_path"] + read_json_params["file"], )

Check *pandas* dataframe:

In [7]:
df.head()

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,3200.00,Ford,Focus
1,E432XX77RUS,1,6500.00,Toyota,Camry
2,7184TT36RUS,1,2100.00,Ford,Focus
3,X582HE161RUS,2,2000.00,Ford,Focus
4,92918M178RUS,1,5700.00,Ford,Focus


* there are values missing from the `Model`, do not do anything with them.

### enrich the dataframe using a `sample` from that dataframe:

* create a sample with `200` new observations with `random_state = 21`:

    * the sample should not have new combinations of the `CarNumber`, `Make`,
      `Model`, so the whole dataset will be consistent in these terms:

In [8]:
sample: pd.DataFrame = df.sample(
    n=200,
    replace=False,  # Add duplicates or not
    random_state=21,
)

    * there are no restrictions on the `Refund` and `Fines`, you can take any
      value from these columns at random and use it towards any `CarNumber`:

In [9]:
sample["Fines"] = df["Fines"].sample(n=200, random_state=21, ).values
sample["Refund"] = df["Refund"].sample(n=200, random_state=21, ).values

Check `sample` *Pandas* dataframe:

In [10]:
sample.head()

,CarNumber,Refund,Fines,Make,Model
445,M0299X197RUS,2,19200.00,Ford,Focus
22,83298C154RUS,2,8594.59,Ford,Focus
93,H957HY161RUS,1,2000.00,Ford,Focus
173,T941CC96RUS,1,2000.00,Ford,Focus
697,H966HY161RUS,1,500.00,Ford,Focus


* concatenate the `sample` with the initial dataframe to a new dataframe `concat_rows`:

In [11]:
concat_rows: pd.DataFrame = pd.concat([df, sample, ], ignore_index=True, )

Check `concat_rows`:

In [12]:
concat_rows.head()

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,3200.00,Ford,Focus
1,E432XX77RUS,1,6500.00,Toyota,Camry
2,7184TT36RUS,1,2100.00,Ford,Focus
3,X582HE161RUS,2,2000.00,Ford,Focus
4,92918M178RUS,1,5700.00,Ford,Focus


### enrich the dataframe `concat_rows` by a new column with the data generated:

* use `np.random.seed(21, )` before generating the years:

In [13]:
np.random.seed(21, )  # Fix random seed

* create a *Pandas* series with the name `Year` using random integers from `1980` to `2019`:

In [14]:
year: pd.Series = pd.Series(np.random.randint(
    1980,
    2019,
    size=concat_rows.shape[0],
), )

* concatenate the series with the dataframe and name it `Year`:

In [15]:
concat_rows = pd.concat([concat_rows, year, ], axis=1, )

In [16]:
concat_rows.rename(columns={0: "Year", }, inplace=True, )

Check updated *Pandas* dataframe:

In [17]:
concat_rows.head()

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995
2,7184TT36RUS,1,2100.00,Ford,Focus,1984
3,X582HE161RUS,2,2000.00,Ford,Focus,2015
4,92918M178RUS,1,5700.00,Ford,Focus,2014


### enrich the dataframe with the data from another dataframe:

* create a new dataframe with the `CarNumber` and their owners:

    * get the most popular surnames in the *US*:

Create a dictionary for `read_json()` method calling:

In [18]:
read_json_params: dict = {"file": "surname.json", "file_path": "../../datasets/", }

Get popular *US* surnames:

In [19]:
sec_df: pd.DataFrame = pd.read_json(read_json_params["file_path"] + read_json_params["file"], )

Check *Pandas* dataframe:

In [20]:
sec_df.head()

,0,1,2
0,NAME,COUNT,RANK
1,ADAMS,427865,42
2,ALLEN,482607,33
3,ALVAREZ,233983,92
4,ANDERSON,784404,15


Set first dataset row as column name row:

In [21]:
sec_df.columns = sec_df.iloc[0]
sec_df = sec_df.drop(0, )

Check *Pandas* dataframe again:

In [22]:
sec_df.head()

,NAME,COUNT,RANK
1,ADAMS,427865,42
2,ALLEN,482607,33
3,ALVAREZ,233983,92
4,ANDERSON,784404,15
5,BAILEY,277845,72


    * create a new series with the surnames from the data you gathered, the count should be equal to the number of unique `CarNumber` using the `sample`:

Calculate count of unique `CarNumber`:

In [23]:
cnt_uniq_car_nums: int = sample["CarNumber"].nunique()

Check it:

In [24]:
cnt_uniq_car_nums

180

Create *Pandas* series:

In [25]:
surnames: pd.Series = sec_df["NAME"].sample(
    n=cnt_uniq_car_nums,
    replace=True,  # Add duplicates or not
    random_state=21,
)

    * create the dataframe owners with `2` columns: `CarNumber`, `Surname`:


In [26]:
owners: pd.DataFrame = pd.DataFrame({"CarNumber": sample["CarNumber"].drop_duplicates().values, "Surname": surnames.values, }, )

Check new dataframe:

In [27]:
owners.head()

,CarNumber,Surname
0,M0299X197RUS,RICHARDSON
1,83298C154RUS,ROSS
2,H957HY161RUS,MORGAN
3,T941CC96RUS,BAILEY
4,H966HY161RUS,LOPEZ


* append `5` more observations to the `owners` dataframe:

Create a new observations:

In [28]:
observations: pd.DataFrame = pd.DataFrame({
    "CarNumber": [
        "83298C228RUS",
        "8329C1337RUS",
        "33328C154RUS",
        "88005C154RUS",
        "77777C154RUS",
    ],
    "Surname": [
        "ERDOGAN",
        "PUTKIN",
        "VODER",
        "SOCKER",
        "DOCKER",
    ],
}, )

Check them:

In [29]:
observations.head()

,CarNumber,Surname
0,83298C228RUS,ERDOGAN
1,8329C1337RUS,PUTKIN
2,33328C154RUS,VODER
3,88005C154RUS,SOCKER
4,77777C154RUS,DOCKER


Append them to `owners`:

In [30]:
owners = pd.concat([owners, observations, ], ignore_index=True, )

Check updated dataframe:

In [31]:
owners.tail()

,CarNumber,Surname
180,83298C228RUS,ERDOGAN
181,8329C1337RUS,PUTKIN
182,33328C154RUS,VODER
183,88005C154RUS,SOCKER
184,77777C154RUS,DOCKER


* delete the dataframe last `20` observations from the `owners` and add `3` new observations:

Delete last `20` rows:

In [32]:
owners = owners.iloc[: -20]

Check updated dataframe:

In [33]:
owners.tail()

,CarNumber,Surname
160,98047H178RUS,WHITE
161,X522OM161RUS,CHAVEZ
162,O136HO197RUS,HOWARD
163,7089H8163RUS,WRIGHT
164,7608EE777RUS,SANDERS


Create a new observations:

In [34]:
new_observations: pd.DataFrame = pd.DataFrame({
    "CarNumber": [
        "8-800-555RUS",
        "212121212RUS",
        "zxc424124RUS",
    ],
    "Surname": [
        "CURSED",
        "OBAMA",
        "TRUMP",
    ],
}, )

Check them:

In [35]:
new_observations.head()

,CarNumber,Surname
0,8-800-555RUS,CURSED
1,212121212RUS,OBAMA
2,zxc424124RUS,TRUMP


Append them to `owners`:

In [36]:
owners = pd.concat([owners, new_observations, ], ignore_index=True, )

Check updated dataframe:

In [37]:
owners.tail()

,CarNumber,Surname
163,7089H8163RUS,WRIGHT
164,7608EE777RUS,SANDERS
165,8-800-555RUS,CURSED
166,212121212RUS,OBAMA
167,zxc424124RUS,TRUMP


* join both dataframes:

    * the new dataframe should have only the `CarNumber` that exist in both dataframes:

In [38]:
res: pd.DataFrame = pd.merge(
    owners,
    concat_rows,
    on="CarNumber",
)

Check result:

In [39]:
res.head()

,CarNumber,Surname,Refund,Fines,Make,Model,Year
0,M0299X197RUS,RICHARDSON,2,67200.00,Ford,Focus,1995
1,M0299X197RUS,RICHARDSON,2,19200.00,Ford,Focus,2005
2,M0299X197RUS,RICHARDSON,1,9600.00,Ford,Focus,2007
3,M0299X197RUS,RICHARDSON,2,19200.00,Ford,Focus,1989
4,83298C154RUS,ROSS,2,8594.59,Ford,Focus,2013


    * the new dataframe should have all the `CarNumber` that exist in both dataframes:

In [40]:
res: pd.DataFrame = pd.merge(
    owners,
    concat_rows,
    on="CarNumber",
    how="outer",
)

Check result:

In [41]:
res.head()

,CarNumber,Surname,Refund,Fines,Make,Model,Year
0,212121212RUS,OBAMA,NaN,NaN,NaN,NaN,NaN
1,704687163RUS,NaN,2.00,1400.00,Ford,Focus,2014.00
2,704787163RUS,NaN,2.00,2800.00,Ford,Focus,2005.00
3,704987163RUS,NaN,2.00,8594.59,Ford,Focus,2014.00
4,705287163RUS,NaN,2.00,2000.00,Ford,Focus,1990.00


    * the new dataframe should have only the `CarNumber` from the `concat_rows` dataframe:

In [42]:
res: pd.DataFrame = pd.merge(
    owners,
    concat_rows,
    on="CarNumber",
    how="right",
)

Check result:

In [43]:
res.head()

,CarNumber,Surname,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,NaN,2,3200.00,Ford,Focus,1989
1,E432XX77RUS,NaN,1,6500.00,Toyota,Camry,1995
2,7184TT36RUS,NaN,1,2100.00,Ford,Focus,1984
3,X582HE161RUS,NaN,2,2000.00,Ford,Focus,2015
4,92918M178RUS,MORGAN,1,5700.00,Ford,Focus,2014


    * the new dataframe should have only the `CarNumber` from the `owners` dataframe:

In [44]:
res: pd.DataFrame = pd.merge(
    owners,
    concat_rows,
    on="CarNumber",
    how="left",
)

Check result:

In [45]:
res.head()

,CarNumber,Surname,Refund,Fines,Make,Model,Year
0,M0299X197RUS,RICHARDSON,2.00,67200.00,Ford,Focus,1995.00
1,M0299X197RUS,RICHARDSON,2.00,19200.00,Ford,Focus,2005.00
2,M0299X197RUS,RICHARDSON,1.00,9600.00,Ford,Focus,2007.00
3,M0299X197RUS,RICHARDSON,2.00,19200.00,Ford,Focus,1989.00
4,83298C154RUS,ROSS,2.00,8594.59,Ford,Focus,2013.00


### create a pivot table from the `concat_rows` dataframe:


In [46]:
pivot_table: pd.DataFrame = pd.pivot_table(
    concat_rows,
    values="Fines",
    index=["Make", "Model", ],
    columns="Year",
    aggfunc="sum",
    fill_value="nan",
)

Check the pivot table:

In [47]:
print(pivot_table, )

Year                   1980      1981      1982      1983      1984      1985  \
Make       Model                                                                
Ford       Focus   89194.59 266783.76 107283.76 147289.17 106000.00 307494.59   
           Mondeo       nan       nan  46200.00       nan       nan       nan   
Skoda      Octavia 13794.59   1900.00   8894.59       nan   1300.00 153594.59   
Toyota     Camry   12000.00       nan   1000.00   8594.59   1000.00       nan   
           Corolla      nan   6800.00       nan  12800.00       nan   4400.00   
Volkswagen Golf    20800.00   8594.59   5000.00    200.00       nan 168000.00   
           Jetta        nan   1000.00       nan       nan       nan   9000.00   
           Passat    900.00  12500.00       nan   1100.00   8594.59       nan   
           Touareg      nan       nan       nan       nan       nan       nan   

Year                   1986     1987     1988      1989  ...      2009  \
Make       Model                  

* save both the `concat_rows` and `owners` dataframes to `.csv` files without an index:

Create a dictionary for saving dataframes to `.csv` files:

In [48]:
to_csv_params: dict = {
    "file_path": "../../datasets/",
    "file_one": "concat_rows.csv",
    "file_two": "owners.csv",
}

Safe *Pandas* dataframes to `.csv` files:

In [49]:
concat_rows.to_csv(to_csv_params["file_path"] + to_csv_params["file_one"], index=False, )

In [50]:
concat_rows.to_csv(to_csv_params["file_path"] + to_csv_params["file_two"], index=False, )